In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://security.ubuntu.com

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/Austin_311_Public_Data.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Austin_311_Public_Data.csv"), sep=",", header=True)
df.show()

+----------------------+--------------------+------------------+------+--------------+--------------+--------------------+-------------+--------------------+------+--------+----------+----------------+
|service_request_number|         description|   method_received|status|  created_date|   closed_date|            location|street_number|         street_name|  city|zip_code|    county|council_dsitrict|
+----------------------+--------------------+------------------+------+--------------+--------------+--------------------+-------------+--------------------+------+--------+----------+----------------+
|           19-00090956|Animal - Proper Care|             Phone|Closed| 3/14/19 15:02|11/23/20 13:41|4609 RIBBECKE AVE...|         4609|            RIBBECKE|AUSTIN|   78721|    TRAVIS|               1|
|           20-00135805|      Tree Issue ROW|     Mobile Device|Closed|  4/7/20 19:06| 11/23/20 0:02|3521 WEST AVE, AU...|         3521|                WEST|AUSTIN|   78705|    TRAVIS|        

In [4]:
# Lets create a view with our DataFrame that counts the occurances of the service request description of the 311 issues. 
# Record the runtime using time.time() method.

df.createOrReplaceTempView('311description')

start_time = time.time()

spark.sql("""select description, count(*) from 311description group by 1""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+---------------------------------------------------+--------+
|description                                        |count(1)|
+---------------------------------------------------+--------+
|Tree Issue ROW/Maintenance (PW)                    |4688    |
|Austin Code - Short Term Rental (STR) Appointment  |848     |
|Park Maintenance - Grounds Plumbing Issues         |2690    |
|Austin Code - Short Term Rental Complaint SR       |9865    |
|Lane/Road Closure Notification                     |9836    |
|ZZZ ARR Missed Recycling                           |20      |
|ATD - Booting Complaint                            |245     |
|Obstruction in ROW                                 |9453    |
|Park Maintenance - Grounds Electrical Issues       |1646    |
|Parking Ticket Complaint                           |407     |
|Parking Machine Issue                              |10756   |
|School Zone Flasher - Timing/Maintenance           |2956    |
|ATD - Shared Micromobility                         |71

In [5]:
# write out the data in parquet format
df.write.parquet('parquet_descriptions',mode='overwrite')



*   click the folder icon on the left and notice that a new folder is present with the same name as your parquet file
*   inside of it you will find 'part_x.parquet' files
*  these are binary files that store your compressed data in columnar format





In [6]:
# read in our new parquet formatted data
p_df=spark.read.parquet('parquet_descriptions')

In [7]:
# a parquet formatted dataframe has all the same methods as a row-based dataframe
# we can convert the dataframe to a view.
p_df.createOrReplaceTempView('p_descriptions')

In [8]:
# Run the same sql as above.  (Note: Due to small datasets it IS possible that times may be very close, however this is not the case in larger sets.)
start_time = time.time()
spark.sql("""select description, count(*) from p_descriptions group by 1 """).show(truncate=False)
print("--- %s seconds ---" % (time.time() - start_time))

+---------------------------------------------------+--------+
|description                                        |count(1)|
+---------------------------------------------------+--------+
|Tree Issue ROW/Maintenance (PW)                    |4688    |
|Austin Code - Short Term Rental (STR) Appointment  |848     |
|Park Maintenance - Grounds Plumbing Issues         |2690    |
|Austin Code - Short Term Rental Complaint SR       |9865    |
|Lane/Road Closure Notification                     |9836    |
|ZZZ ARR Missed Recycling                           |20      |
|ATD - Booting Complaint                            |245     |
|Obstruction in ROW                                 |9453    |
|Park Maintenance - Grounds Electrical Issues       |1646    |
|Parking Ticket Complaint                           |407     |
|Parking Machine Issue                              |10756   |
|School Zone Flasher - Timing/Maintenance           |2956    |
|ATD - Shared Micromobility                         |71